<a href="https://colab.research.google.com/github/isinghtara/-Building-LLM-Prompt-And-Stroutput-Chains-With-LCEL/blob/main/RAG_With_Cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community

In [37]:
import os
import getpass

os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API key: ")

Enter your Cohere API key: ··········


### RAG Step

In [38]:
!pip install pypdf
from langchain_community.document_loaders import PyPDFLoader

In [39]:
#documetns Loading
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf
loader = PyPDFLoader("EMPLOYEE_AGREEMENT.pdf")
pages = loader.load()
print(len(pages))

--2025-12-23 10:54:26--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206079 (201K) [application/octet-stream]
Saving to: ‘EMPLOYEE_AGREEMENT.pdf.2’

EMPLOYEE_AGREEMENT. 100%[===================>] 201.25K  --.-KB/s    in 0.02s   

2025-12-23 10:54:26 (11.1 MB/s) - ‘EMPLOYEE_AGREEMENT.pdf.2’ saved [206079/206079]

13


In [40]:
#chunking

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

133


In [41]:
#Embadding
#!pip install langchain-cohere --quiet
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)

In [42]:
#Vector Database

!pip install chromadb --quiet

In [43]:
from langchain_community.vectorstores import Chroma

In [49]:
emp_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="emp_rules_db"
                          )
emp_rules_db.persist()

In [53]:
retriever = emp_rules_db.as_retriever()
result=retriever.invoke("What is the policy for sick leaves")
result

[Document(metadata={'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page_label': '2', 'title': 'EMPLOYEE AGREEMENT', 'page': 1, 'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'producer': 'Skia/PDF m123'}, page_content='sick days and personal absence days as authorized by the Company for its other employees.\nARTICLE 6\nFringe Benefits\nEmployee shall be entitled to participate in and receive benefits from all of the Company’s employee\nbenefit plans that are now, or in the future may be, maintained by the'),
 Document(metadata={'creationdate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '2', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'total_pages': 13, 'title': 'EMPLOYEE A